In [1]:
import os
import pandas as pd
import numpy as np
import numba

df_main=pd.read_csv('data/normalized_filtered_user_listening.csv')
# df=pd.read_csv('data/user_listening.csv')

In [2]:
df=df_main[0:10000]

In [3]:
matrix=df.pivot(index='track_id',columns='user_id',values='normalized_playcount')
# matrix=df.pivot(index='track_id',columns='user_id',values='playcount')
arr=np.array(matrix)

In [4]:
arr[100]

array([nan, nan, nan, ..., nan, nan, nan])

In [5]:
print('Unique user count :' , len(df['user_id'].unique()))
print('Unique track count :' , len(df['track_id'].unique()))

#filled elements
known_values=list(zip(np.where(~np.isnan(arr))[0],
        np.where(~np.isnan(arr))[1]))

#randomly select 30% as testing set
lucky_draw=set(np.random.choice(range(len(known_values)),
                 size=int(len(known_values)*0.3),
                 p=[1/len(known_values)]*len(known_values)))

unlucky_draw=[i for i in range(len(known_values)) if i not in lucky_draw]
testing_idx=(np.where(~np.isnan(arr))[0][list(lucky_draw)],
np.where(~np.isnan(arr))[1][list(lucky_draw)])
training_idx=(np.where(~np.isnan(arr))[0][list(unlucky_draw)],
np.where(~np.isnan(arr))[1][list(unlucky_draw)])

#train test split
mask=np.ones(arr.shape)
mask[testing_idx]=np.nan
arr_train=np.multiply(arr,mask)

Unique user count : 1877
Unique track count : 5077


## Koren Neighborhood Model


In [6]:
#use numba to dramatically boost the speed of linear algebra
@numba.njit
def koren_neighbor_epoch(arr,miu,b_u,b_i,
                         similarity_matrix,
                         w_ij,alpha,lambda_,top_k):
    
    #initialize
    error=0
    
    #only iterate known ratings
    for i in range(arr.shape[0]):
        for u in range(arr.shape[1]):
            r_ui=arr[i,u]
            
            #another way to identify nan
            #r_ui!=r_ui
            if np.isnan(r_ui):
                continue
                
            #find top k neighbor based upon similarity matrix
            rated_items=np.where(~np.isnan(arr[:,u]))[0]
            similarities=similarity_matrix[i][rated_items]
            top_k_neighbors=np.argsort(similarities)[-top_k:]
            N_k_i_u=np.array([
                    rated_items[neighbor] for neighbor in top_k_neighbors])
            
            #compute error
            if len(N_k_i_u)!=0:
                deviation=arr[:,u][N_k_i_u]-miu-b_u[u]-b_i[N_k_i_u]
                weighted_sum=(w_ij[i][N_k_i_u]).T@deviation
                epsilon_ui=(r_ui-miu-b_u[u]-b_i[i]-weighted_sum).item()
                error+=epsilon_ui**2
            else:
                epsilon_ui=(r_ui-miu-b_u[u]-b_i[i]).item()
                error+=epsilon_ui**2

            #update baseline
            b_u[u]+=alpha*(epsilon_ui-lambda_*b_u[u])
            b_i[i]+=alpha*(epsilon_ui-lambda_*b_i[i])
            
            #only update weights when there are similar items                
            if len(N_k_i_u)!=0:
                N_k_i_u_norm_sqrt=N_k_i_u.shape[0]**0.5                
                w_ij[i][N_k_i_u]=w_ij[i][N_k_i_u]+alpha*(
                    epsilon_ui*deviation/N_k_i_u_norm_sqrt-lambda_*w_ij[
                        i][N_k_i_u])
                w_ij[N_k_i_u][:,i]=w_ij[i][N_k_i_u]
                                        
    return error,b_u,b_i,w_ij

In [7]:
#koren weighted neighborhood model
def koren_neighbor(arr,similarity_matrix,
                   miu_init=None,b_u_init=[],
                   b_i_init=[],w_ij_init=[],
                   alpha=0.005,lambda_=0.02,
                   tau=0.0001,top_k=10,
                   max_iter=20,diagnosis=True):

    #initialize
    stop=False
    counter=0
    sse=None
    
    #global mean
    if not miu_init:       
        miu=arr[~np.isnan(arr)].mean()
    else:
        miu=miu_init
        
    #user baseline
    if len(b_u_init)==0:
        b_u=np.zeros(arr.shape[1])
    else:
        b_u=b_u_init
    
    #item baseline
    if len(b_i_init)==0:
        b_i=np.zeros(arr.shape[0])
    else:
        b_i=b_i_init
        
    #weighted neighbor
    if len(w_ij_init)==0:
        w_ij=np.zeros((arr.shape[0],arr.shape[0]))
        w_ij.fill(0.001)
    else:
        w_ij=w_ij_init
    
    #gradient descent
    while not stop:
        
        error,b_u,b_i,w_ij=koren_neighbor_epoch(
                         arr,miu,b_u,b_i,
                         similarity_matrix,
                         w_ij,alpha,lambda_,top_k)

        counter+=1
        
        #maximum number of epoch
        if counter>=max_iter:
            stop=True
            if diagnosis:
                print('Not converged.',
                      'Consider increase number of iterations or tolerance')
                
        #use sum of squared error to determine if converged
        sse_prev=sse
        sse=error
        if sse_prev and abs(sse/sse_prev-1)<=tau:
            stop=True
            if diagnosis:
                print(f'{counter} iterations to reach convergence\n')

    return b_u,b_i,w_ij

In [8]:
#obtain msd similarity matrix
@numba.njit
def get_msd_similarity_matrix(arr):

    similarity_matrix=np.zeros((arr.shape[1],arr.shape[1]))
    for u in range(arr.shape[1]):
        for v in range(u+1,arr.shape[1]):
            
            #self correlation distorts knn selection
            if u==v:
                continue

            #compute msd first then eliminate nan
            I_uv=np.square(arr[:,u]-arr[:,v])
            valid_I_uv=I_uv[~np.isnan(I_uv)]

            #avoid the case where two users havent rated any items in common
            if len(valid_I_uv)>0:
                msd=1/(valid_I_uv.sum()/len(valid_I_uv)+1)
            else:
                msd=0

            #symmetric matrix
            similarity_matrix[u,v]=msd
            similarity_matrix[v,u]=msd
            
    return similarity_matrix

In [9]:
#compute item similarity matrix
similarity_matrix=get_msd_similarity_matrix(arr_train.T)

In [10]:
#initialize
num_of_latent_factors=40
max_num_of_epoch=150
learning_rate=0.01
lagrange_multiplier=0.02
tolerance=0.01
top_k=10
#koren neighbor
b_u,b_i,w_ij=koren_neighbor(arr_train,
                   similarity_matrix,
                   alpha=learning_rate,
                   lambda_=lagrange_multiplier,
                   tau=tolerance,top_k=10,
                   max_iter=max_num_of_epoch)

67 iterations to reach convergence



In [11]:
#compute global mean
miu=arr_train[~np.isnan(arr_train)].mean()

#compute deviation
deviation=arr_train-np.repeat(
            b_u.reshape(1,-1),
            arr_train.shape[0],axis=0)+np.repeat(
            b_i.reshape(-1,1),
    arr_train.shape[1],axis=1)-miu

#set nan to zero for dot product
deviation[np.isnan(deviation)]=0

#find top k neighbors for each item
top_k_neighbors=np.argsort(similarity_matrix,axis=0)[-top_k:]

#identify the col index of top k neighbors
col=top_k_neighbors.flatten()

#identify the row index of each item
row=np.array([j for i in range(arr_train.shape[0]) for j in [i]*top_k])

#create weights matrix
weights=np.zeros(w_ij.shape)

#only keep weights of top k neighbors
weights[(row,col)]=w_ij[(row,col)]

#compute influence from weighted neighbors
weighted_neighbors=weights@deviation

In [12]:
#matrix completion
output=miu+np.repeat(
            b_u.reshape(1,-1),
            arr_train.shape[0],axis=0)+np.repeat(
            b_i.reshape(-1,1),
    arr_train.shape[1],axis=1)+weighted_neighbors

In [13]:
#find top k neighbors for each item,
top_k=10
top_k_neighbors=np.argsort(similarity_matrix,axis=0)[-top_k:]

In [14]:
top_k_neighbors[:,1]

array([ 920, 1633, 2407,  786, 3335, 3654,  484, 4337,  556, 3776],
      dtype=int64)

In [15]:
b_u.shape

(1877,)

In [16]:
similarity_matrix[100]

array([0., 0., 0., ..., 0., 0., 0.])

In [17]:
output[:,100][::-1]

array([0.39831661, 0.40135909, 0.45241777, ..., 0.40745266, 0.49127164,
       0.47545638])

In [26]:
def filter_already_rated_items(user_id, output, df,arr, N=10):
    """
    Filter already rated items and recommend top-N items to the user.

    Args:
        user_id (int): ID of the user (e.g., 100 for the 100th user).
        output (np.ndarray): Predicted ratings for all items (shape: num_items).
        df (pd.DataFrame): Initial DataFrame containing user-item ratings.
        N (int): Number of top recommendations to generate (default: 10).

    Returns:
        list: Top-N recommended item indices (excluding already rated items).
    """
    # Get the row index corresponding to the user ID
    user_row_index = user_id  # Assuming user IDs match row indices

    # Extract the user's actual ratings from the initial DataFrame
    user_ratings = df.loc[user_row_index].values

    # Filter out items that the user has already rated
    unrated_items = np.where(np.isnan(arr[user_id]))[0]
    
    # Sort unrated items by predicted ratings (highest to lowest)
    sorted_indices = np.argsort(output[unrated_items,user_id])[::-1]
    
    # Get the top-N unrated items (excluding already rated items)
    top_N_items = sorted_indices[:N]

    return top_N_items

# Example usage:
# user_id: ID of the 100th user
# output: Predicted ratings for all items (shape: num_items)
# df: Initial DataFrame containing user-item ratings
top_N_recommendations = filter_already_rated_items(204, output, df,arr, N=10)
top_N_recommendations


array([1373,  366, 1352, 1246,  959,  750,  256,  525,  388, 1721],
      dtype=int64)

In [28]:
# Gather the results
big_array = np.concatenate([filter_already_rated_items(i, output, df,arr, N=10) for i in range(output.shape[1])])

# Find the top 10 recurring values
unique_values, counts = np.unique(big_array, return_counts=True)
sorted_indices = np.argsort(counts)[::-1]
top_10_values = unique_values[sorted_indices[:10]]
top_10_counts = counts[sorted_indices[:10]]

# Reveal the secrets
for value, count in zip(top_10_values, top_10_counts):
    print(f"{value} repeats {count} times")

256 repeats 1448 times
366 repeats 1341 times
388 repeats 1311 times
525 repeats 1167 times
1721 repeats 1126 times
1372 repeats 1023 times
1351 repeats 1014 times
1245 repeats 952 times
750 repeats 866 times
958 repeats 852 times
